In [1]:
# Temperature_Forecast_Project


import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import cross_val_score,GridSearchCV
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')


df=pd.read_csv(r'C:\ProgramData\temperature.csv')
df.head()

,station,Date,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,...,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
0,1.0,30-06-2013,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,...,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996,29.1,21.2
1,2.0,30-06-2013,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,...,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500,30.5,22.5
2,3.0,30-06-2013,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,...,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664,31.1,23.9
3,4.0,30-06-2013,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,...,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844,31.7,24.3
4,5.0,30-06-2013,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,...,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246,31.2,22.5


In [2]:
df.shape

(7752, 25)

In [3]:
# The dataset comprises of 7752 rows and 25 columns.

In [4]:
# Exploratory Data Analysis

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7752 entries, 0 to 7751
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   station           7750 non-null   float64
 1   Date              7750 non-null   object 
 2   Present_Tmax      7682 non-null   float64
 3   Present_Tmin      7682 non-null   float64
 4   LDAPS_RHmin       7677 non-null   float64
 5   LDAPS_RHmax       7677 non-null   float64
 6   LDAPS_Tmax_lapse  7677 non-null   float64
 7   LDAPS_Tmin_lapse  7677 non-null   float64
 8   LDAPS_WS          7677 non-null   float64
 9   LDAPS_LH          7677 non-null   float64
 10  LDAPS_CC1         7677 non-null   float64
 11  LDAPS_CC2         7677 non-null   float64
 12  LDAPS_CC3         7677 non-null   float64
 13  LDAPS_CC4         7677 non-null   float64
 14  LDAPS_PPT1        7677 non-null   float64
 15  LDAPS_PPT2        7677 non-null   float64
 16  LDAPS_PPT3        7677 non-null   float64


In [5]:
# Most columns are of float type except one of object type

In [6]:
# Checking missing values

df.isnull().sum()

station              2
Date                 2
Present_Tmax        70
Present_Tmin        70
LDAPS_RHmin         75
LDAPS_RHmax         75
LDAPS_Tmax_lapse    75
LDAPS_Tmin_lapse    75
LDAPS_WS            75
LDAPS_LH            75
LDAPS_CC1           75
LDAPS_CC2           75
LDAPS_CC3           75
LDAPS_CC4           75
LDAPS_PPT1          75
LDAPS_PPT2          75
LDAPS_PPT3          75
LDAPS_PPT4          75
lat                  0
lon                  0
DEM                  0
Slope                0
Solar radiation      0
Next_Tmax           27
Next_Tmin           27
dtype: int64

In [7]:
# In comparision to number of rows in dataset,missing values are less,so we can drop missing values rows

In [8]:
# Dropping missing values rows

df.dropna(axis=0,inplace=True)

In [9]:
df.isnull().sum()

station             0
Date                0
Present_Tmax        0
Present_Tmin        0
LDAPS_RHmin         0
LDAPS_RHmax         0
LDAPS_Tmax_lapse    0
LDAPS_Tmin_lapse    0
LDAPS_WS            0
LDAPS_LH            0
LDAPS_CC1           0
LDAPS_CC2           0
LDAPS_CC3           0
LDAPS_CC4           0
LDAPS_PPT1          0
LDAPS_PPT2          0
LDAPS_PPT3          0
LDAPS_PPT4          0
lat                 0
lon                 0
DEM                 0
Slope               0
Solar radiation     0
Next_Tmax           0
Next_Tmin           0
dtype: int64

In [10]:
# # Statistical summary

df.describe()

,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,...,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
count,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,...,7588.000000,7588.000000,7588.000000,7588.000000,7588.00000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000
mean,13.014101,29.748366,23.195809,56.724969,88.360823,29.620128,23.511786,7.094097,62.492606,0.368510,...,0.480738,0.275007,0.265373,37.544792,126.99142,61.918136,1.259755,5343.724208,30.241526,22.910820
std,7.217858,2.967401,2.400880,14.626559,7.199456,2.943496,2.342579,2.177034,33.686158,0.262260,...,1.743327,1.146087,1.179661,0.050428,0.07922,54.323529,1.372748,429.782561,3.111807,2.482256
min,1.000000,20.000000,11.300000,19.794666,58.936283,17.624954,14.272646,2.882580,-13.603212,0.000000,...,0.000000,0.000000,0.000000,37.456200,126.82600,12.370000,0.098500,4329.520508,17.400000,11.300000
25%,7.000000,27.800000,21.600000,45.960243,84.203724,27.673756,22.086820,5.675358,37.206201,0.146546,...,0.000000,0.000000,0.000000,37.510200,126.93700,28.700000,0.271300,5001.485717,28.200000,21.300000
50%,13.000000,29.900000,23.400000,55.023199,89.784122,29.709537,23.758249,6.547838,56.898324,0.315706,...,0.000000,0.000000,0.000000,37.550700,126.99500,45.716000,0.618000,5441.987305,30.400000,23.100000
75%,19.000000,32.000000,24.800000,67.115099,93.742725,31.711109,25.155660,8.028960,84.235666,0.574174,...,0.017735,0.007855,0.000017,37.577600,127.04200,59.832400,1.767800,5729.485840,32.600000,24.600000
max,25.000000,37.600000,29.900000,98.524734,100.000153,38.542255,29.619342,21.857621,213.414006,0.967277,...,21.621661,15.841235,16.655469,37.645000,127.13500,212.335000,5.178200,5992.895996,38.900000,29.800000


In [11]:
# Above description clearly shows that there are no missing values now in dataset

In [12]:
# Splitting date into day,month ,year


df["Day"] =df['Date'].str.split('-').str[0].astype(int)
df["Month"]=df['Date'].str.split('-').str[1].astype(int)
df["Year"] =df['Date'].str.split('-').str[2].astype(int)

In [13]:
# Dropping Date column as it is unnecessary now

df=df.drop(['Date'],axis=1)

In [14]:
# Converting float type columns to numeric type

for i in df:
    df[i] = df[i].astype(int)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7588 entries, 0 to 7749
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   station           7588 non-null   int32
 1   Present_Tmax      7588 non-null   int32
 2   Present_Tmin      7588 non-null   int32
 3   LDAPS_RHmin       7588 non-null   int32
 4   LDAPS_RHmax       7588 non-null   int32
 5   LDAPS_Tmax_lapse  7588 non-null   int32
 6   LDAPS_Tmin_lapse  7588 non-null   int32
 7   LDAPS_WS          7588 non-null   int32
 8   LDAPS_LH          7588 non-null   int32
 9   LDAPS_CC1         7588 non-null   int32
 10  LDAPS_CC2         7588 non-null   int32
 11  LDAPS_CC3         7588 non-null   int32
 12  LDAPS_CC4         7588 non-null   int32
 13  LDAPS_PPT1        7588 non-null   int32
 14  LDAPS_PPT2        7588 non-null   int32
 15  LDAPS_PPT3        7588 non-null   int32
 16  LDAPS_PPT4        7588 non-null   int32
 17  lat               7588 non-null  

In [16]:
# Now all columns have been converted to integer type

In [17]:

X=df.drop(['Next_Tmax','Next_Tmin'],axis=1)
y=df.get(['Next_Tmax','Next_Tmin'])

In [18]:
# Checking skewness in features

X.skew().sort_values()

LDAPS_RHmax        -0.843460
LDAPS_Tmin_lapse   -0.570672
Solar radiation    -0.524335
Present_Tmin       -0.344937
Present_Tmax       -0.255961
LDAPS_Tmax_lapse   -0.213990
Month              -0.174856
Day                -0.010718
station            -0.004178
lat                 0.000000
LDAPS_CC3           0.000000
LDAPS_CC4           0.000000
LDAPS_CC1           0.000000
LDAPS_CC2           0.000000
Year                0.013575
lon                 0.078093
LDAPS_RHmin         0.300909
LDAPS_LH            0.663223
LDAPS_WS            1.513559
DEM                 1.720833
Slope               1.804018
LDAPS_PPT1          5.671074
LDAPS_PPT2          6.129018
LDAPS_PPT3          6.913513
LDAPS_PPT4          7.159558
dtype: float64

In [19]:
# Removing skewness

from sklearn.preprocessing import power_transform
z = power_transform(X[0:])
data_new= pd.DataFrame(z,columns=X.columns)
X = data_new


In [20]:
# Checking skewness in features

X.skew().sort_values()

Day                -0.217762
station            -0.206291
Solar radiation    -0.182375
LDAPS_RHmax        -0.120487
Month              -0.084425
Present_Tmax       -0.036564
LDAPS_Tmin_lapse   -0.022731
LDAPS_RHmin        -0.018459
Present_Tmin       -0.014642
LDAPS_Tmax_lapse   -0.011812
LDAPS_WS           -0.001669
lat                 0.000000
LDAPS_CC4           0.000000
LDAPS_CC2           0.000000
LDAPS_CC1           0.000000
LDAPS_CC3           0.000000
LDAPS_LH            0.019865
DEM                 0.038818
lon                 0.078093
Year                0.409648
Slope               0.643425
LDAPS_PPT1          2.178040
LDAPS_PPT2          2.752291
LDAPS_PPT3          3.598170
LDAPS_PPT4          3.717460
dtype: float64

In [21]:

# Scaling the dataset and normalizing feature variables

from sklearn.preprocessing import StandardScaler 

scale = StandardScaler()
X_features=X
X= scale.fit_transform(X)

In [22]:
# Evaluation of models

#Training model with DecisionTreeRegressor and finding the best state,r2_score

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

model_dt = DecisionTreeRegressor()

score_s=0
state=0
for i in range(0,25):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state =i)
    model_dt.fit(X_train, y_train)
    y_pred_dt = model_dt.predict(X_test)
    score=r2_score(y_test,y_pred_dt)
    if score>score_s:
        score_s=score
        state=i
        
print('best random_state : ',state)
print('best r2 score : ',score_s)

best random_state :  5
best r2 score :  0.740314852871004


In [23]:
# finding mean_squared_error,rmse  for DecisionTreeRegressor

mse=mean_squared_error(y_test,y_pred_dt)
rmse=np.sqrt(mse)

rmse

1.516488211424233

In [24]:
# Evaluation of models

#Training model with LinearRegression and finding the best state,r2_score

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

model_lr = LinearRegression()

score_s=0
state=0
for i in range(0,25):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state =i)
    model_lr.fit(X_train, y_train)
    y_pred_lr = model_lr.predict(X_test)
    score=r2_score(y_test,y_pred_lr)
    if score>score_s:
        score_s=score
        state=i
        
print('best random_state : ',state)
print('best r2 score : ',score_s)

best random_state :  7
best r2 score :  0.7919297125937352


In [25]:
# finding mean_squared_error,rmse  for LinearRegression

mse=mean_squared_error(y_test,y_pred_lr)
rmse=np.sqrt(mse)

rmse

1.3288962344293433

In [26]:
# Evaluation of models

#Training model with RandomForestRegressor and finding the best state,r2_score

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

model_rfr = RandomForestRegressor()

score_s=0
state=0
for i in range(0,10):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state =i)
    model_rfr.fit(X_train, y_train)
    y_pred_rfr = model_rfr.predict(X_test)
    score=r2_score(y_test,y_pred_rfr)
    if score>score_s:
        score_s=score
        state=i
        
print('best random_state : ',state)
print('best r2 score : ',score_s)

best random_state :  7
best r2 score :  0.8804863008745256


In [27]:
# finding mean_squared_error,rmse  for RandomForestRegressor

mse=mean_squared_error(y_test,y_pred_rfr)
rmse=np.sqrt(mse)

rmse

1.0115940429270787

In [28]:
# Evaluation of models

#Training model with KNeighborsRegressor and finding the best state,r2_score

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

model_knr = KNeighborsRegressor()

score_s=0
state=0
for i in range(0,25):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state =i)
    model_knr.fit(X_train, y_train)
    y_pred_knr = model_knr.predict(X_test)
    score=r2_score(y_test,y_pred_knr)
    if score>score_s:
        score_s=score
        state=i
        
print('best random_state : ',state)
print('best r2 score : ',score_s)

best random_state :  24
best r2 score :  0.8232887730051577


In [29]:
# finding mean_squared_error,rmse  for KNeighborsRegressor

mse=mean_squared_error(y_test,y_pred_knr)
rmse=np.sqrt(mse)

rmse

1.2100806275470604

In [ ]:
# Clearly RandomForestRegressor model looks good enough to predict primary_fuel and capacity_mw

In [30]:
# HyperParameterTuning with  RandomForestRegressor

from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state =12)

estimator = RandomForestRegressor()
param_grid = { 
            "n_estimators"      : [10,20,30],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False],
            }

In [31]:
grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)

grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_split': [2, 4, 8],
                         'n_estimators': [10, 20, 30]})

In [32]:
grid.best_score_

0.8856904230438755

In [33]:
grid.best_params_

{'bootstrap': False,
 'max_features': 'log2',
 'min_samples_split': 2,
 'n_estimators': 30}

In [34]:
grid.score(X_test,y_test)

0.890985388534449

In [ ]:
#Clearly Hyperparameter tuning improved model performance.

In [36]:

grid.fit(X_train,y_train)
y_pred_rfr=grid.predict(X_test)
y_pred_rfr[0:20]

array([[31.13333333, 22.43333333],
       [33.66666667, 26.2       ],
       [34.46666667, 25.46666667],
       [26.63333333, 22.        ],
       [29.2       , 19.76666667],
       [27.8       , 20.43333333],
       [31.93333333, 22.13333333],
       [28.33333333, 20.7       ],
       [29.73333333, 22.2       ],
       [30.        , 26.13333333],
       [28.06666667, 22.23333333],
       [30.03333333, 24.03333333],
       [34.13333333, 26.56666667],
       [28.66666667, 22.33333333],
       [28.06666667, 22.56666667],
       [25.53333333, 21.63333333],
       [27.5       , 20.73333333],
       [30.63333333, 23.83333333],
       [28.1       , 19.9       ],
       [30.93333333, 23.8       ]])

In [37]:
y_test[0:20]

,Next_Tmax,Next_Tmin
4244,31,23
7038,33,26
5821,35,26
2747,25,23
2993,28,20
4681,28,20
3328,33,22
1535,29,21
624,31,23
5238,30,26


In [38]:
score=r2_score(y_test,y_pred_rfr)
score

0.8882922124045877

In [ ]:
# Exporting the model through pickle

import pickle
filename='TempForecastPred.pkl'
pickle.dump(grid,open(filename,'wb'))

In [ ]:
# Conclusion 

# Clearly we can see that the actual value and predicted values are very close to each other,Hence RandomForestRegressor is
# a good choice for predicting temperature forecast